In [19]:
import numpy as np
import open3d as o3d
import pathlib
import math
import itertools

## Loading in Point cloud data

In [20]:
# Load ply file
path = "./data/P002 2022-01-25 01_24_48.ply"

if pathlib.Path(path).exists():
    pcd = o3d.io.read_point_cloud(path)
    print('PLY file loaded')
else:
    raise FileNotFoundError("FILE NOT FOUND, TRY AGAIN!")

PLY file loaded


In [21]:
# PLY file info
print(dir(pcd))

['HalfEdgeTriangleMesh', 'Image', 'LineSet', 'PointCloud', 'RGBDImage', 'TetraMesh', 'TriangleMesh', 'Type', 'Unspecified', 'VoxelGrid', '__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'cluster_dbscan', 'colors', 'compute_convex_hull', 'compute_mahalanobis_distance', 'compute_mean_and_covariance', 'compute_nearest_neighbor_distance', 'compute_point_cloud_distance', 'covariances', 'create_from_depth_image', 'create_from_rgbd_image', 'crop', 'detect_planar_patches', 'dimension', 'estimate_covariances', 'estimate_normals', 'estimate_point_covariances', 'farthest_point_down_sample', 'get_axis_aligned_bounding_box', 'get_center', 'get_geometry_type', 'get_max_bound', 

In [22]:
print('Shape of points', np.asarray(pcd.points).shape)
print('Shape of colors', np.asarray(pcd.colors).shape)

Shape of points (251759, 3)
Shape of colors (251759, 3)


# Preprocessing

In [23]:
class PreProcessing:
    def __init__(self, voxelSize=0.01, itera=10000):
        self.voxelSize = voxelSize
        self.itera = itera
    
    def minMaxView(self, pcd):
        '''
        used to visualise the min/max points in each respective axes
        '''
        
        # Colors:
        RED = [1., 0., 0.]
        GREEN = [0., 1., 0.]
        BLUE = [0., 0., 1.]
        YELLOW = [1., 1., 0.]
        MAGENTA = [1., 0., 1.]
        CYAN = [0., 1., 1.]
        
        # create 3d coordinate system
        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
        
        # Get max and min points of each axis x, y and z:
        x_max, x_min = max(pcd.points, key=lambda x: x[0]), min(pcd.points, key=lambda x: x[0])
        y_max, y_min = max(pcd.points, key=lambda x: x[1]), min(pcd.points, key=lambda x: x[1])
        z_max, z_min = max(pcd.points, key=lambda x: x[2]), min(pcd.points, key=lambda x: x[2])

        # floor cleaning
        pcd = self.bounding_box_minor(pcd)
        self.visualise(pcd, windowName="bounding box minor")

        # paint the floor as 0
        epsilon = 0.05
        points = np.asarray(pcd.points)
        indices = np.where(np.abs(points[:, 2] < epsilon))[0]
        
        pcd = pcd.select_by_index(indices, invert=True)
        # pcd.colors = o3d.utility.Vector3dVector(pcd)
        # colors = np.zeros(points.shape)
        # colors[indices] = BLUE
        
        self.visualise(pcd, points, windowName="z index floor")
        
        # print(colors) 
        
        
        # positions = [x_max, y_max, z_max, x_min, y_min, z_min]
        # geometries = [pcd, origin]
        # colors = [RED, GREEN, BLUE, MAGENTA, YELLOW, CYAN]
        # for i in range(len(positions)):
        #     # Create a sphere mesh:
        #     sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
        #     # move to the point position:
        #     sphere.translate(np.asarray(positions[i]))
        #     # add color:
        #     sphere.paint_uniform_color(np.asarray(colors[i]))
        #     # compute normals for vertices or faces:
        #     sphere.compute_vertex_normals()
        #     # add to geometry list to display later:
        #     geometries.append(sphere)

        # Display:
        # o3d.visualization.draw_geometries(pcd)
        
        # ########################
        # # Define a threshold:
        # THRESHOLD = 0.1

        # # Get the max value along the y-axis:
        # y_max = max(pcd.points, key=lambda x: x[1])[1]

        # # Get the original points color to be updated:
        # pcd_colors = np.asarray(pcd.colors)

        # # Number of points:
        # n_points = pcd_colors.shape[0]

        # # update color:
        # for i in range(n_points):
        #     # if the current point is aground point:
        #     if pcd.points[i][1] >= y_max - THRESHOLD:
        #         pcd_colors[i] = RED  # color it green

        # pcd.colors = o3d.utility.Vector3dVector(pcd_colors)

        # # Display:
        # o3d.visualization.draw_geometries([pcd, origin])
        
        return pcd
        
            
    def downsample_clean(self, pcd):
        '''
        most of the cleaning is done here
        '''
         
        # create 3d coordinate system
        origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
        
        # reduce total number of points
        downsampled = pcd.voxel_down_sample(self.voxelSize) 
        self.visualise(origin, pcd)
        
        # outlier cleaning
        cl, ind = downsampled.remove_statistical_outlier(nb_neighbors=100, std_ratio=2.0)
        cleaned_pcd = downsampled.select_by_index(ind)
        
        # removing majority floor
        plane_model, inliers = cleaned_pcd.segment_plane(distance_threshold=0.02, ransac_n=5, num_iterations=self.itera)
        inlier_cloud = cleaned_pcd.select_by_index(inliers)
        inlier_cloud.paint_uniform_color([1,0,0])
        outlier_cloud = cleaned_pcd.select_by_index(inliers, invert=True)
        o3d.visualization.draw_geometries([outlier_cloud, inlier_cloud])

        return outlier_cloud
    
    def bounding_box_major(self, pcd):
        '''
        applying bounding box method removing majority of unwanted points
        '''
        
        points = np.asarray(pcd.points)
        
        min_x, min_y, min_z = points.min(axis=0)[:3]
        max_x, max_y, max_z = points.max(axis=0)[:3]

        padding = 0.53
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(padding*min_x, padding*min_y ,min_z ), max_bound=(padding*max_x, padding*max_y, max_z))

        # Select points within the bounding box
        pcd_in_bbox = pcd.crop(bbox)
        
        return pcd_in_bbox

    def bounding_box_minor(self, pcd):
        # Create bounding box:
        bounds = [[-math.inf, math.inf], [-math.inf, math.inf], [0, 0.0458]]  # set the bounds
        
        bounding_box_points = list(itertools.product(*bounds))  # create limit points
        bounding_box = o3d.geometry.AxisAlignedBoundingBox.create_from_points(
            o3d.utility.Vector3dVector(bounding_box_points))  # create bounding box object
        
        # Crop the point cloud using the bounding box:
        pcd_croped = pcd.crop(bounding_box)
        dists = np.asarray(pcd.compute_point_cloud_distance(pcd_croped))
        indices = np.where(dists > 0.000001)[0]
        pcd_cropped_inv = pcd.select_by_index(indices)

        # Display the cropped point cloud:
        self.visualise(pcd_croped, windowName="pcd cropped")
        self.visualise(pcd_cropped_inv, windowName="pcd cropped inverse")
        
        return pcd_cropped_inv
    
    def visualise(self, *args, windowName="Point Cloud"):
        # drawing
        o3d.visualization.draw_geometries(
            [*args], 
            window_name=windowName
        )

process = PreProcessing()
pcd_in_bbox = process.bounding_box_major(pcd)

inlier = process.downsample_clean(pcd_in_bbox)
process.visualise(inlier)

cropped = process.minMaxView(inlier)
process.visualise(cropped)

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


TypeError: draw_geometries(): incompatible function arguments. The following argument types are supported:
    1. (geometry_list: List[open3d.cpu.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False) -> None
    2. (geometry_list: List[open3d.cpu.pybind.geometry.Geometry], window_name: str = 'Open3D', width: int = 1920, height: int = 1080, left: int = 50, top: int = 50, point_show_normal: bool = False, mesh_show_wireframe: bool = False, mesh_show_back_face: bool = False, lookat: numpy.ndarray[numpy.float64[3, 1]], up: numpy.ndarray[numpy.float64[3, 1]], front: numpy.ndarray[numpy.float64[3, 1]], zoom: float) -> None

Invoked with: [PointCloud with 19640 points., array([[-0.40546346, -0.14839208,  1.05951929],
       [ 0.15353656, -0.15314204,  0.99951929],
       [-0.06133839, -0.01839203,  1.07951933],
       ...